In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [2]:
def jump_diffusion(K_1):
    # Parameters
    sigma = 0.15
    r = 0.05
    T = 1
    S_0 = 1
    #K_1 = 0.9
    K_3 = 2*S_0-K_1
    K_2 = (K_1+K_3)/2
    mu_up = 3.04
    mu_down = 3.08
    p_up = 0.34
    lambda_ = 0.1
    N = 1000
    n = 50000
    kappa = (p_up * mu_up)/(mu_up - 1)+ (1-p_up)*mu_down/(mu_down + 1)+1
    delta_t = T/n
    for i in range(n):
        res = []
        S = [S_0]
        for j in range(N):
    
            Z = np.random.normal(0, 1)
            u = np.random.uniform(0,1)
            if u <= lambda_ * delta_t:
                #hasjump = 1
                # now sample from the jump distribution
                u_2 = np.random.uniform(0,1)
                if u_2 <= (1-p_up):
                    x = 1/mu_down * np.log(u/(1-p_up))
                    jump = np.exp(x)
                else:
                    x = -1/mu_up * np.log(1-(u-(1-p_up))/p_up)
                    jump = np.exp(x)
            else:
                #hasjump = 0
                jump = 1
            s_j = S[-1] * (1+ (r-lambda_*kappa)*delta_t + sigma*np.sqrt(delta_t)*Z + (jump-1))
            S.append(s_j)
        #print(len(S))
        if S[-1] <= K_1:
            payout = 0
        elif S[-1] <= K_2:
            payout = S[-1]-K_1
        elif S[-1] <= K_3:
            payout = K_3-S[-1]
        else:  
            payout = 0
        res.append(payout*np.exp(-r*T))

    return [(K_3-K_1)/2, np.mean(res)]

In [3]:

from pyspark.sql import SparkSession

# Initialize a Spark session
spark = SparkSession.builder.appName("OptionPricingSimulation").getOrCreate()

# Parallelize the K_1 values for distribution across the cluster
K_list = [0.6, 0.65, 0.7, 0.75, 0.8, 0.85, 0.9]
K_rdd = spark.sparkContext.parallelize(K_list)

# Define a wrapper function for jump_diffusion to be used in map operation
def jump_diffusion_wrapper(K_1):
    # Your existing jump_diffusion function code here
    return jump_diffusion(K_1)

# Apply the simulation function to each element in the RDD
results = K_rdd.map(jump_diffusion_wrapper).collect()

# The 'results' variable now contains the output from all distributed simulations
# Proceed with any aggregation or plotting as in the original notebook

spark.stop()

ModuleNotFoundError: No module named 'pyspark'

In [ ]:
x = []
y = []
K_list = [0.6, 0.65, 0.7, 0.75, 0.8, 0.85, 0.9]
for K_1 in K_list:
    x.append(jump_diffusion(K_1)[0])
    y.append(jump_diffusion(K_1)[1])
plt.plot(x,  y, label='European Butterfly Spread', linestyle='--')

# Adding title
plt.title('European Butterfly Spread')
# Labeling the axes
plt.xlabel('(K3-K1)/2')
plt.ylabel('Price')

# Adding legend
plt.legend()

# Displaying the plot
plt.show()

# comment:
The price increases with the spread width, which is close to positive linear based on this graph. This makes sense because a wider spread between the strike prices K1 and K3 increases the range within which the butterfly spread can be profitable at expiration, leading to a higher price for the position. The more the potential for profit within that range, the higher the option's price. 



In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
def jump_diffusion(K_1):
    # Parameters
    sigma = 0.15
    r = 0.05
    T = 1
    S_0 = 1
    #K_1 = 0.9
    K_3 = 2*S_0-K_1
    K_2 = (K_1+K_3)/2
    mu_up = 3.04
    mu_down = 3.08
    p_up = 0.34
    lambda_ = 0.1
    N = 1000
    n = 50000
    kappa = (p_up * mu_up)/(mu_up - 1)+ (1-p_up)*mu_down/(mu_down + 1)+1
    delta_t = T/n
    for i in range(n):
        res = []
        S = [S_0]
        for j in range(N):
    
            Z = np.random.normal(0, 1)
            u = np.random.uniform(0,1)
            if u <= lambda_ * delta_t:
                #hasjump = 1
                # now sample from the jump distribution
                u_2 = np.random.uniform(0,1)
                if u_2 <= (1-p_up):
                    x = 1/mu_down * np.log(u/(1-p_up))
                    jump = np.exp(x)
                else:
                    x = -1/mu_up * np.log(1-(u-(1-p_up))/p_up)
                    jump = np.exp(x)
            else:
                #hasjump = 0
                jump = 1
            s_j = S[-1] * (1+ (r-lambda_*kappa)*delta_t + sigma*np.sqrt(delta_t)*Z + (jump-1))
            S.append(s_j)
        #print(len(S))
        if S[-1] <= K_1:
            payout = 0
        elif S[-1] <= K_2:
            payout = S[-1]-K_1
        elif S[-1] <= K_3:
            payout = K_3-S[-1]
        else:  
            payout = 0
        res.append(payout*np.exp(-r*T))

    return [(K_3-K_1)/2, np.mean(res)]

In [ ]:
x = []
y = []
K_list = [0.6, 0.65, 0.7, 0.75, 0.8, 0.85, 0.9]
for K_1 in K_list:
    x.append(jump_diffusion(K_1)[0])
    y.append(jump_diffusion(K_1)[1])
plt.plot(x,  y, label='European Butterfly Spread', linestyle='--')

# Adding title
plt.title('European Butterfly Spread')
# Labeling the axes
plt.xlabel('(K3-K1)/2')
plt.ylabel('Price')

# Adding legend
plt.legend()

# Displaying the plot
plt.show()

# comment:
The price increases with the spread width, which is close to positive linear based on this graph. This makes sense because a wider spread between the strike prices K1 and K3 increases the range within which the butterfly spread can be profitable at expiration, leading to a higher price for the position. The more the potential for profit within that range, the higher the option's price. 

